In [1]:
import numpy as np
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic
import pandas as pd

data = pd.read_csv('water.txt', sep="\t")
data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [2]:
def calculate(dataframe):
    mean = np.mean(dataframe)
    mean_std = dataframe.std(ddof=1)/np.sqrt(len(dataframe))
    print _tconfint_generic(mean, mean_std,
    len(dataframe) - 1,
    0.05, 'two-sided')

In [3]:
calculate(data['mortality'])
calculate(data[data['location'] == 'South']['mortality'])

calculate(data[data['location'] == 'South']['hardness'])
calculate(data[data['location'] == 'North']['hardness'])

(1476.0833413552848, 1572.2117406119285)
(1320.1517462936238, 1433.463638321761)
(53.467198692036106, 86.07126284642544)
(21.42248728572426, 39.37751271427574)


In [4]:
from statsmodels.stats.proportion import samplesize_confint_proportion
from statsmodels.stats.proportion import proportion_confint

In [5]:
wilson_interval = proportion_confint(1, 50, method = 'wilson')
print wilson_interval[0]

0.003539259271646236


In [6]:
n_samples = int(np.ceil(samplesize_confint_proportion(0.02, 0.01)))
print n_samples

753


In [7]:
from statsmodels.compat.python import lzip, range
import numpy as np
from scipy import stats, optimize
from sys import float_info

from statsmodels.stats.base import AllPairsResults
from statsmodels.tools.sm_exceptions import HypothesisTestWarning


def proportion_confint_custom(count, nobs, alpha=0.05, method='normal'):
    pd_index = getattr(count, 'index', None)
    if pd_index is not None and hasattr(pd_index, '__call__'):
        # this rules out lists, lists have an index method
        pd_index = None
    count = np.asarray(count)
    nobs = np.asarray(nobs)

    q_ = count * 1. / nobs
    alpha_2 = 0.5 * alpha

    if method == 'normal':
        std_ = np.sqrt(q_ * (1 - q_) / nobs)
        dist = stats.norm.isf(alpha / 2.) * std_
        ci_low = q_ - dist
        ci_upp = q_ + dist

    elif method == 'binom_test':
        # inverting the binomial test
        def func(qi):
            return stats.binom_test(q_ * nobs, nobs, p=qi) - alpha
        if count == 0:
            ci_low = 0
        else:
            ci_low = optimize.brentq(func, float_info.min, q_)
        if count == nobs:
            ci_upp = 1
        else:
            ci_upp = optimize.brentq(func, q_, 1. - float_info.epsilon)

    elif method == 'beta':
        ci_low = stats.beta.ppf(alpha_2, count, nobs - count + 1)
        ci_upp = stats.beta.isf(alpha_2, count + 1, nobs - count)

        if np.ndim(ci_low) > 0:
            ci_low[q_ == 0] = 0
            ci_upp[q_ == 1] = 1
        else:
            ci_low = ci_low if (q_ != 0) else 0
            ci_upp = ci_upp if (q_ != 1) else 1

    elif method == 'agresti_coull':
        crit = stats.norm.isf(alpha / 2.)
        nobs_c = nobs + crit**2
        q_c = (count + crit**2 / 2.) / nobs_c
        std_c = np.sqrt(q_c * (1. - q_c) / nobs_c)
        dist = crit * std_c
        ci_low = q_c - dist
        ci_upp = q_c + dist

    elif method == 'wilson':
        crit = stats.norm.isf(alpha / 2.)
        crit2 = crit**2
        denom = 1 + crit2 / nobs
        center = (q_ + crit2 / (2 * nobs)) / denom
        dist = crit * np.sqrt(q_ * (1. - q_) / nobs + crit2 / (4. * nobs**2))
        dist /= denom
        ci_low = center - dist
        ci_upp = center + dist

    # method adjusted to be more forgiving of misspellings or incorrect option name
    elif method[:4] == 'jeff':
        ci_low, ci_upp = stats.beta.interval(1 - alpha, count + 0.5,
                                             nobs - count + 0.5)

    else:
        raise NotImplementedError('method "%s" is not available' % method)

    if pd_index is not None and np.ndim(ci_low) > 0:
        import pandas as pd
        if np.ndim(ci_low) == 1:
            ci_low = pd.Series(ci_low, index=pd_index)
            ci_upp = pd.Series(ci_upp, index=pd_index)
        if np.ndim(ci_low) == 2:
            ci_low = pd.DataFrame(ci_low, index=pd_index)
            ci_upp = pd.DataFrame(ci_upp, index=pd_index)

    return ci_low, ci_upp

In [8]:
normal_interval = proportion_confint_custom(1, 50, method = 'normal')
print normal_interval[0]

-0.018805307081790987


# 1. Уточнение правила трёх сигм

Не забудьте, что при построении интервала остаточную вероятностную массу нужно делить между двумя хвостами распределения (если вы не понимаете, что имеется в виду, попробуйте вернуться к видео "Интервальные оценки с помощью квантилей" и обратите внимание, квантили какого порядка используются при построении 95% интервала).
Не рекомендуется использовать "аналоговые" таблицы квантилей — в них легко могут быть опечатки, числа там могут быть слишком сильно округлены, да и в целом это способ прошлого века.

In [9]:
from scipy.stats import norm
rv = norm(0,1)
rv.ppf(0.9985)

2.9677379253417944

# 5. Снижение вероятности инфаркта при приёме аспирина

* Имеющиеся у нас выборки бинарны, поэтому работать с ними нужно методами для долей — tconfint не пойдёт.
* Проверьте, что вы взяли разность долей в том порядке, в котором этого требует условие задания.
* В этом задании часто ошибка возникает из-за невнимательности — проверьте, что вы не перепутали числа.

In [10]:
p = 104.0/11037

p_1 = 189.0/11034

print p_1 - p

0.007706023976


# 6. Доверительный интервал для снижения вероятности инфаркта при приёме аспирина

* Функция для построения доверительного интервала определена в видео "Доверительные интервалы для двух долей" и соответствующем ноутбуке.
* На вход ей неоходимо подавать выборки, поэтому вам необходимо сгенерировать две выборки из нулей и единиц, в которой единицы будут соответствовать людям, у которых случился инфаркт, а нули — людям, у которых за 5 лет он не произошёл.

In [11]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

def proportions_confint_diff_ind(sum_sample1, sum_sample2, len_sample1, len_sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum_sample1) / len_sample1
    p2 = float(sum_sample2) / len_sample2
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len_sample1 + p2 * (1 - p2)/ len_sample2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len_sample1 + p2 * (1 - p2)/ len_sample2)
    
    return (left_boundary, right_boundary)

In [12]:
proportions_confint_diff_ind(189, 104, 11034, 11037, 0.05)

(0.004687750675049439, 0.010724297276960124)

# 7. Отношение шансов инфаркта при приёме аспирина и плацебо

Для ответа на этот вопрос не нужно даже генерировать выборки — достаточно рассчитать шансы по формуле, приведённой в условии, и поделить их друг на друга в правильном порядке.
Проверьте, что в вашей цепочке вычислений нет промежуточных округлений (например, что вы не округляете шансы перед тем, как их делить друг на друга).

In [13]:
print  (p_1 / (1 - p_1)) / (p / (1 - p))

1.83205394191


# 8. Доверительный интервал для отношения шансов

В этом задании есть элемент случайности, поэтому, чтобы получить такой же ответ, какой прописан в грейдере, вам нужно в точности вопроизвести последовательность действий из условия задания.

* При создании исходных выборок сначала записывайте в них единицы, а потом нули.
* Установите random_seed один раз перед началом генерации бутстреп-псевдовыборок. Помните, что установка random_seed должна делаться в той же ячейке ноутбука, что и генерация выборок.
* Генерируйте псевдовыборки с помощью функции get_bootstrap_samples из ноутбука урока "Доверительные интервалы на основе бутстрепа". Чтобы сгенерировать все необходимые псевдовыборки, вам понадобится два вызова этой функции.
* Если вы использовали в вашем решении код из ноутбука урока "Доверительные интервалы на основе бутстрепа", убедитесь, что в нём не осталось кусков, относящихся к задаче, решавшейся в этом ноутбуке (например, что вы не считаете на псевдовыборках разность медиан вместо отношения шансов).

In [15]:
np.random.seed(0)

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

p_plac = np.append(np.repeat(1.0, 104), np.repeat(0.0, 11037-104))

p_true = np.append(np.repeat(1.0, 189), np.repeat(0.0, 11034-189))

def shance(a,b):
    return (a/b)/(1-a/b)

def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

placebo = map(lambda x:shance(sum(x),len(x)), get_bootstrap_samples(p_plac, 1000))
aspirin = map(lambda x:shance(sum(x),len(x)), get_bootstrap_samples(p_true, 1000))

vect_ot_chanse=map(lambda x: x[1]/x[0], zip(placebo, aspirin))

print stat_intervals(vect_ot_chanse, 0.05)

[1.44419465 2.34321168]


# TEST 2

# 1. Опрос в ресторане

Если у вас не получается понять, что от вас требуется, попробуйте переформулировать вопрос в терминах проверки гипотез:”отличается ли уровень стресса сотрудников ресторана от среднего?”

Также обратите внимание на тот вид альтернативы, которую будете использовать - односторонняя или двусторонняя альтернатива?

In [16]:
stats.binom_test(67, 100, 0.75, alternative = 'greater')

0.9724054358699054

In [18]:
import pandas as pd
dataset=pd.read_csv("pines.txt",delimiter="\t")

dataset.head()

,sn,we
0,200.0,8.8
1,199.3,10.0
2,193.6,22.4
3,167.7,35.6
4,183.9,45.4


In [28]:
from scipy.stats import binned_statistic_2d
bins = [0.0, 40.0, 80.0, 120.0, 160.0, 200.0]
result = binned_statistic_2d(dataset['sn'], dataset['we'], None, 'count', bins=[bins,bins])
print np.mean(result.statistic)
print len(dataset)/25.0

23.36
23.36


In [34]:
array = result.statistic.reshape(25)
array_stat = [len(dataset)/25.0] * 25
stats.chisquare(array, array_stat, ddof = 24)

Power_divergenceResult(statistic=150.58904109589042, pvalue=nan)